<a href="https://colab.research.google.com/github/nakul-krishnakumar/lang-dubbing-pipeline/blob/main/nakul_sarvam_assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENGLISH TO MALAYALAM AUDIO DUBBING PIPELINE

## Installation & Setup of Dependencies

In [ ]:
!pip cache purge

In [ ]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.8 MB/s eta 0:00:00


In [ ]:
import os
import json
from google.colab import userdata
from IPython.display import Video, Audio, display
from pprint import pprint

In [ ]:
# All file directories
FILE_DIR = "/content"
INPUT_VIDEO_PATH = os.path.join(FILE_DIR, "input_data/input_video.mp4")
INPUT_AUDIO_PATH = os.path.join(FILE_DIR, "input_data/input_audio.wav")
PREPROC_PATH = os.path.join(FILE_DIR, "preprocessed_audio")
PREPROC_VOCALS_PATH = os.path.join(FILE_DIR, "preprocessed_audio/vocals.wav")
PREPROC_ACCOMP_PATH = os.path.join(FILE_DIR, "preprocessed_audio/accompaniment.wav")
TRANS_TEXT_PATH = os.path.join(FILE_DIR, "text_data")
OUTPUT_AUDIO_PATH = os.path.join(FILE_DIR, "output_data")
OUTPUT_AUDIO_FILE = os.path.join(OUTPUT_AUDIO_PATH, "output_audio.wav")
RAW_OUTPUT_AUDIO_FILE = os.path.join(OUTPUT_AUDIO_PATH, "raw_output_audio.wav")

## Downloading the Source Video from Youtube

In [ ]:
!yt-dlp -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]' -o "input_data/input_video.mp4" "https://www.youtube.com/watch?v=eMc2dRfYZuE"

[youtube] Extracting URL: https://www.youtube.com/watch?v=eMc2dRfYZuE
[youtube] eMc2dRfYZuE: Downloading webpage
[youtube] eMc2dRfYZuE: Downloading tv client config
[youtube] eMc2dRfYZuE: Downloading player 59b252b9-main
[youtube] eMc2dRfYZuE: Downloading tv player API JSON
[youtube] eMc2dRfYZuE: Downloading ios player API JSON
[youtube] eMc2dRfYZuE: Downloading m3u8 information
[info] eMc2dRfYZuE: Downloading 1 format(s): 625+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 50
[download] Destination: input_data/input_video.f625.mp4
[download] 100% of  209.87MiB in 00:00:15 at 13.64MiB/s
[download] Destination: input_data/input_video.f140.m4a
[download] 100% of    4.07MiB in 00:00:00 at 7.87MiB/s
[Merger] Merging formats into "input_data/input_video.mp4"
Deleting original file input_data/input_video.f625.mp4 (pass -k to keep)
Deleting original file input_data/input_video.f140.m4a (pass -k to keep)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%ls

drive/  input_data/  sample_data/


In [ ]:
# If yt-dlp does not work, use shutil to move the input video from drive to working directory
import shutil

# Paths
source_path = "/content/drive/MyDrive/input_data/input_video.mp4"
destination_path = "/content/input_data/input_video.mp4"

os.makedirs("input_data", exist_ok=True)
# Move the file
shutil.move(source_path, destination_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/input_data/input_video.mp4'

In [ ]:
# Function to check if a file exists at a given path
def do_file_exists(file_path):
    if os.path.exists(file_path):
        print("Input video exists at: ", file_path)
    else:
        print("Input video does not exist")

In [ ]:
do_file_exists(INPUT_VIDEO_PATH) # Check if the input video exists

Input video exists at:  /content/input_data/input_video.mp4


In [ ]:
# Display the input video
Video(INPUT_VIDEO_PATH, width=640, height=360, embed=True)

## Extracting Audio from the Source Video

In [ ]:
# # Extracting audio from the input video
!ffmpeg -i input_data/input_video.mp4 -ac 1 -ar 16000 -vn input_data/input_audio.wav -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
# Check if the converted audio file exists
do_file_exists(INPUT_AUDIO_PATH)

Input video exists at:  /content/input_data/input_audio.wav


In [ ]:
# Display the extracted audio
Audio(INPUT_AUDIO_PATH, autoplay=False)

## Splitting the Audio into Vocals & BGM

In [ ]:
!pip install spleeter

In [ ]:
from spleeter.separator import Separator

def sep_audio_vocals(input_audio_path, output_dir):
    """
    This function is used to separate the vocals and accompaniment from the input audio

    Args:
    input_audio_path: Path to the input audio file
    output_dir: Directory where the separated audio files will be saved
    """
    # Initialize with 2 stems (vocals + accompaniment)
    separator = Separator('spleeter:2stems')

    # Process the extracted audio file
    separator.separate_to_file(
        input_audio_path,
        output_dir,
        filename_format='{instrument}.{codec}'
    )

In [ ]:
os.makedirs(PREPROC_PATH, exist_ok=True)

sep_audio_vocals(INPUT_AUDIO_PATH, PREPROC_PATH) # separate vocals and accompaniments from input audio

In [ ]:
from IPython.display import Audio, display

# Display separated tracks
display(
    Audio(PREPROC_VOCALS_PATH, autoplay=False),
    Audio(PREPROC_ACCOMP_PATH, autoplay=False)
)


## Speech-to-Text Conversion

### Using AssemblyAI SpeechModel

In [ ]:
!pip install assemblyai

In [ ]:
import assemblyai as aai

aai.settings.api_key = userdata.get("ASSEMBLY_AI_API_KEY")

transcriber = aai.Transcriber()

config = aai.TranscriptionConfig(speech_model=aai.SpeechModel.slam_1)

transcript = transcriber.transcribe(PREPROC_VOCALS_PATH, config)

if transcript.status == aai.TranscriptStatus.error:
    print(f"Transcription failed: {transcript.error}")
    exit(1)

print(f" \nFull Transcript: \n\n{transcript.text}")


 
Full Transcript: 

Language is a unique fingerprint of a culture, right? So when we were thinking about AI for Bharat, we were really genuinely thinking, what is AI for Bharat? And a computer vision algorithm will also have to be optimized for images in India. But the customization that you need for languages is significantly higher. The origin story is quite interesting because this was the time when deep learning was taking off. It was sort of like the previous generation of AI. this one being generative AI. And we started teaching courses on deep learning together. We realized that the area is moving from being somewhat theoretically driven to being more practically driven, doing things at scale, building out these models was interesting. So we started teaching. We got a lot of people who were interested to then build this out practically. And that's when the idea of AI for Bharat started, right? So let's build these systems for real. Often they require a lot of investment, so we 

In [ ]:
stt_text = transcript.text

os.makedirs(TRANS_TEXT_PATH, exist_ok=True)
with open(os.path.join(TRANS_TEXT_PATH, "stt_text.txt"), "w") as f:
    f.write(stt_text)

In [ ]:
with open(os.path.join(TRANS_TEXT_PATH, "stt_text.txt"), "r") as f:
    stt_text = f.read()

# pprint(stt_text)

In [ ]:
import re

def preprocess_text(stt_text):
    """
    This function is used to do manual text preprocessing for better audio output.
    """
    sentences = re.findall(r'[^.!?;]+[.!?;]?', stt_text)

    preproc_sent = []
    for sentence in sentences:
        sentence = sentence.replace("AI for Bharat", "AI4Bharat")

        preproc_sent.append(sentence)

    return preproc_sent

In [ ]:
sentences = preprocess_text(stt_text)
# sentences

### Using Sarvam Batch API ( Did not convert all the words, missed some )
I had tried Batch API first, but it simply ignores some words and the sentences at the end of some of the transcripts.
You can refer the outputs of the below codeblocks to notice it.
Note the last transcript entry where the transcription is only till 'become' but there is more audio to be transcripted

In [ ]:
!pip install requests pandas pydub
!pip install azure-storage-file-datalake aiofiles aiohttp requests # for batch-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.4/207.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 19.6 MB/s eta 0:00:00


In [ ]:
import asyncio
import aiofiles
import requests
import json
from urllib.parse import urlparse
from azure.storage.filedatalake.aio import DataLakeDirectoryClient, FileSystemClient
from azure.storage.filedatalake import ContentSettings
import mimetypes
import logging
from pprint import pprint
import os

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Configuration
API_SUBSCRIPTION_KEY = userdata.get("SARVAM_SUBSCRIPTION_KEY")
LANGUAGE_CODE = "en-IN"

In [ ]:
class SarvamClient:
    def __init__(self, url: str):
        self.account_url, self.file_system_name, self.directory_name, self.sas_token = (
            self._extract_url_components(url)
        )
        self.lock = asyncio.Lock()
        print(f"Initialized SarvamClient with directory: {self.directory_name}")

    def update_url(self, url: str):
        self.account_url, self.file_system_name, self.directory_name, self.sas_token = (
            self._extract_url_components(url)
        )
        print(f"Updated URL to directory: {self.directory_name}")

    def _extract_url_components(self, url: str):
        parsed_url = urlparse(url)
        account_url = f"{parsed_url.scheme}://{parsed_url.netloc}".replace(
            ".blob.", ".dfs."
        )
        path_components = parsed_url.path.strip("/").split("/")
        file_system_name = path_components[0]
        directory_name = "/".join(path_components[1:])
        sas_token = parsed_url.query
        return account_url, file_system_name, directory_name, sas_token

    async def upload_files(self, local_file_paths, overwrite=True):
        print(f"Starting upload of {len(local_file_paths)} files")
        async with DataLakeDirectoryClient(
            account_url=f"{self.account_url}?{self.sas_token}",
            file_system_name=self.file_system_name,
            directory_name=self.directory_name,
            credential=None,
        ) as directory_client:
            tasks = []
            for path in local_file_paths:
                file_name = path.split("/")[-1]
                tasks.append(
                    self._upload_file(directory_client, path, file_name, overwrite)
                )
            results = await asyncio.gather(*tasks, return_exceptions=True)
            print(
                f"Upload completed for {sum(1 for r in results if not isinstance(r, Exception))} files"
            )

    async def _upload_file(
        self, directory_client, local_file_path, file_name, overwrite=True
    ):
        try:
            async with aiofiles.open(local_file_path, mode="rb") as file_data:
                mime_type = mimetypes.guess_type(local_file_path)[0] or "audio/wav"
                file_client = directory_client.get_file_client(file_name)
                data = await file_data.read()
                await file_client.upload_data(
                    data,
                    overwrite=overwrite,
                    content_settings=ContentSettings(content_type=mime_type),
                )
                print(f"✅ File uploaded successfully: {file_name}")
                print(f"   Type: {mime_type}")
                return True
        except Exception as e:
            print(f"❌ Upload failed for {file_name}: {str(e)}")
            return False

    async def list_files(self):
        print("\n📂 Listing files in directory...")
        file_names = []
        async with FileSystemClient(
            account_url=f"{self.account_url}?{self.sas_token}",
            file_system_name=self.file_system_name,
            credential=None,
        ) as file_system_client:
            async for path in file_system_client.get_paths(self.directory_name):
                file_name = path.name.split("/")[-1]
                async with self.lock:
                    file_names.append(file_name)
        print(f"Found {len(file_names)} files:")
        for file in file_names:
            print(f"   📄 {file}")
        return file_names

    async def download_files(self, file_names, destination_dir):
        print(f"\n⬇️ Starting download of {len(file_names)} files to {destination_dir}")
        async with DataLakeDirectoryClient(
            account_url=f"{self.account_url}?{self.sas_token}",
            file_system_name=self.file_system_name,
            directory_name=self.directory_name,
            credential=None,
        ) as directory_client:
            tasks = []
            for file_name in file_names:
                tasks.append(
                    self._download_file(directory_client, file_name, destination_dir)
                )
            results = await asyncio.gather(*tasks, return_exceptions=True)
            print(
                f"Download completed for {sum(1 for r in results if not isinstance(r, Exception))} files"
            )

    async def _download_file(self, directory_client, file_name, destination_dir):
        try:
            file_client = directory_client.get_file_client(file_name)
            download_path = f"{destination_dir}/{file_name}"
            async with aiofiles.open(download_path, mode="wb") as file_data:
                stream = await file_client.download_file()
                data = await stream.readall()
                await file_data.write(data)
            print(f"✅ Downloaded: {file_name} -> {download_path}")
            return True
        except Exception as e:
            print(f"❌ Download failed for {file_name}: {str(e)}")
            return False

In [ ]:
async def initialize_job():
    print("\n🚀 Initializing job...")
    url = "https://api.sarvam.ai/speech-to-text/job/init"
    headers = {"API-Subscription-Key": API_SUBSCRIPTION_KEY}
    response = requests.post(url, headers=headers)
    print("\nInitialize Job Response:")
    print(f"Status Code: {response.status_code}")
    print("Response Body:")
    pprint(response.json() if response.status_code == 202 else response.text)

    if response.status_code == 202:
        return response.json()
    return None


async def check_job_status(job_id):
    print(f"\n🔍 Checking status for job: {job_id}")
    url = f"https://api.sarvam.ai/speech-to-text/job/{job_id}/status"
    headers = {"API-Subscription-Key": API_SUBSCRIPTION_KEY}
    response = requests.get(url, headers=headers)
    print("\nJob Status Response:")
    print(f"Status Code: {response.status_code}")
    print("Response Body:")
    pprint(response.json() if response.status_code == 200 else response.text)

    if response.status_code == 200:
        return response.json()
    return None


async def start_job(job_id, language_code=LANGUAGE_CODE):
    print(f"\n▶️ Starting job: {job_id}")
    url = "https://api.sarvam.ai/speech-to-text/job"
    headers = {
        "API-Subscription-Key": API_SUBSCRIPTION_KEY,
        "Content-Type": "application/json",
    }
    data = {"job_id": job_id, "job_parameters": {
        "language_code": language_code,
        "with_timestamps": True,
        "with_diarization": False
    }}
    print("\nRequest Body:")
    pprint(data)

    response = requests.post(url, headers=headers, data=json.dumps(data))
    print("\nStart Job Response:")
    print(f"Status Code: {response.status_code}")
    print("Response Body:")
    pprint(response.json() if response.status_code == 200 else response.text)

    if response.status_code == 200:
        return response.json()
    return None

In [ ]:
from pydub import AudioSegment
import os

def split_audio_into_chunks(input_audio_path, chunk_duration_ms=3 * 60 * 1000, output_dir="chunks"):
    """
    Splits an audio file into chunks of given duration and saves them to a directory.

    input_audio_path: Path to the input audio file
    chunk_duration_ms: Duration of each chunk in milliseconds (default 3 minutes)
    output_dir: Directory where chunks will be saved

    return: List of paths to the chunk files
    """
    audio = AudioSegment.from_file(input_audio_path)
    total_length = len(audio)
    os.makedirs(output_dir, exist_ok=True)

    chunk_paths = []
    for i in range(0, total_length, chunk_duration_ms):
        chunk = audio[i:i + chunk_duration_ms]
        chunk_filename = os.path.join(output_dir, f"chunk_{i // chunk_duration_ms + 1}.wav")
        chunk.export(chunk_filename, format="wav")
        chunk_paths.append(chunk_filename)
        print(f"Saved: {chunk_filename}")

    print(f"\n✅ Total {len(chunk_paths)} chunks created.")
    return chunk_paths


In [ ]:
split_audio_into_chunks()

In [ ]:
async def main():
    print("\n=== Starting Speech-to-Text Processing for All Chunks ===")

    # Step 1: Split the audio into chunks
    chunk_paths = split_audio_into_chunks(PREPROC_VOCALS_PATH, output_dir=os.path.join(FILE_DIR, "preprocessed_audio"))
    print(f"🔀 Total Chunks Created: {len(chunk_paths)}")

    for idx, chunk_path in enumerate(chunk_paths):
        print(f"\n=== Processing Chunk {idx + 1}/{len(chunk_paths)}: {os.path.basename(chunk_path)} ===")

        # Step 2: Initialize a new job for this chunk
        job_info = await initialize_job()
        if not job_info:
            print("❌ Job initialization failed for this chunk")
            continue

        job_id = job_info["job_id"]
        input_storage_path = job_info["input_storage_path"]
        output_storage_path = job_info["output_storage_path"]

        # Step 3: Upload chunk to input storage
        client = SarvamClient(input_storage_path)
        await client.upload_files([chunk_path])
        print("📤 Chunk uploaded")

        # Step 4: Wait for job to be ACCEPTED
        print("⏳ Waiting for job to be ACCEPTED...")
        while True:
            job_status = await check_job_status(job_id)
            if not job_status:
                print("❌ Failed to get job status")
                break

            if job_status["job_state"] == "Accepted":
                print("✅ Job is now ACCEPTED")
                break
            elif job_status["job_state"] == "Failed":
                print("❌ Job failed during validation")
                break
            else:
                print(f"⏳ Still waiting... Current state: {job_status['job_state']}")
                await asyncio.sleep(5)

        # Step 5: Start the job
        job_start_response = await start_job(job_id)
        if not job_start_response:
            print("❌ Failed to start job")
            continue
        print("🚀 Job started")

        # Step 6: Wait for job to COMPLETE
        print("⏳ Waiting for job to complete...")
        while True:
            job_status = await check_job_status(job_id)
            if not job_status:
                print("❌ Failed to get job status")
                break

            if job_status["job_state"] == "Completed":
                print("✅ Job completed")
                break
            elif job_status["job_state"] == "Failed":
                print("❌ Job failed during execution")
                break
            else:
                print(f"⏳ Still processing... Current state: {job_status['job_state']}")
                await asyncio.sleep(10)

        # Step 7: Download the result
        print(f"📥 Downloading result for Chunk {idx + 1}")
        client.update_url(output_storage_path)

        files = await client.list_files()
        chunk_result_dir = os.path.join(TRANS_TEXT_PATH, f"chunk_{idx + 1}")
        os.makedirs(chunk_result_dir, exist_ok=True)

        await client.download_files(files, destination_dir=chunk_result_dir)
        print(f"✅ Files downloaded to: {chunk_result_dir}")

    print("\n=== All Chunks Processed ===")

if __name__ == "__main__":
    await main()

#### Preprocessing Converted Text

In [ ]:
# Define paths
chunk1_path = os.path.join(TRANS_TEXT_PATH, "chunk_1", "0.json")
chunk2_path = os.path.join(TRANS_TEXT_PATH, "chunk_2", "0.json")


# Load JSON data from the file
with open(chunk1_path, 'r') as f:
    chunk1 = json.load(f)

with open(chunk2_path, 'r') as f:
    chunk2 = json.load(f)

pprint(chunk1)
pprint(chunk2)

# Notice that both the chunks have few errors
# chunk1 has an unwanted initial '0.0' entry in both start_time and end_time.
# chunk2 has start_time and end_time starting from 0.0, instead they should start from chunk1's last end_time.

In [ ]:
import os
import json

def refine_text_chunks(chunk1, chunk2):
    """
    This function is used to manually refining the converted text chunks
    """

    # Refine chunk_1
    # Remove the first element of start_time_seconds and end_time_seconds
    chunk1["timestamps"]["start_time_seconds"] = chunk1["timestamps"]["start_time_seconds"][1:]
    chunk1["timestamps"]["end_time_seconds"] = chunk1["timestamps"]["end_time_seconds"][1:]

    # Refine chunk_2
    chunk1_end_time = chunk1["timestamps"]["end_time_seconds"][-1]

    chunk2["timestamps"]["start_time_seconds"] = [
        t + chunk1_end_time for t in chunk2["timestamps"]["start_time_seconds"]
    ]

    chunk2["timestamps"]["end_time_seconds"] = [
        t + chunk1_end_time for t in chunk2["timestamps"]["end_time_seconds"]
    ]

    return chunk1, chunk2

def merge_text_chunks(chunk1, chunk2):
    """
    This function is used to merge the refined text chunks into a single list
    """
    ref_chunk1, ref_chunk2 = refine_text_chunks(chunk1, chunk2)

    segments = []
    for start, end, text in zip(
        ref_chunk1["timestamps"]["start_time_seconds"],
        ref_chunk1["timestamps"]["end_time_seconds"],
        ref_chunk1["timestamps"]["words"]
    ):
        segments.append({
            "start": start,
            "end": end,
            "text": text,
            "duration": end - start
        })

    for start, end, text in zip(
        ref_chunk2["timestamps"]["start_time_seconds"],
        ref_chunk2["timestamps"]["end_time_seconds"],
        ref_chunk2["timestamps"]["words"]
    ):
        segments.append({
            "start": start,
            "end": end,
            "text": text,
            "duration": end - start
        })

    # print("============================== SEGMENTS ======================================")
    # pprint(segments)

    return segments

def preprocess_text(segments):
    """
    This function is used to do manual text preprocessing for better audio output.
    """
    for seg in segments:
        seg["text"] = seg["text"].replace("AI for Bharat", "AI4Bharat")
        seg["text"] = seg["text"].replace(",", " ") # audio generated using bulbulv2 had a weird stop at ','

    return segments

In [ ]:
segments = merge_text_chunks(chunk1, chunk2) # merge texts chunks into one

In [ ]:
segments = preprocess_text(segments)
segments

## Sarvam Client API Setup

In [ ]:
!pip install sarvamai
!pip install --upgrade httpx
!pip install pydub

In [ ]:
import librosa
import soundfile as sf
from pydub import AudioSegment
import base64
import io
from typing import Dict, List
from sarvamai import SarvamAI

class SarvamClientAPI():
    def __init__(self, api_subscription_key):
        self.client = SarvamAI(api_subscription_key=api_subscription_key,)
        pass

    def translate_text(
            self,
            sentences,
            source_lang: str,
            target_lang: str,
            gender: str = "Male",
            mode: str = "code-mixed",
            output_script: str = "spoken-form-in-native",
            enable_preprocessing: bool = False
        ) -> List[Dict]:
        """
        This function is used to translate the given text from source to target language.

        Returns:
            List[Dict]: A list of dictionaries, where each dictionary includes:
        """

        # English to Malayalam text preprocessing using mayura-api
        translated_texts = []
        # print(sentences)
        for i, sentence in enumerate(sentences):
            # print(sentence)
            if sentence and sentence.strip():  # skip None or empty/whitespace-only strings
                response = self.client.text.translate(
                    input=sentence.strip(),  # strip leading/trailing whitespace
                    source_language_code=source_lang,
                    target_language_code=target_lang,
                    speaker_gender=gender,
                    mode=mode,
                    output_script=output_script,
                    enable_preprocessing=enable_preprocessing
                )

                print(f"Processed batch {i + 1} ...")
                translated_texts.append(response.translated_text)


                # translated_texts.append({
                #     "start": seg["start"],
                #     "end": seg["end"],
                #     "text": response.translated_text,
                #     "duration": seg['duration']
                # })
        return translated_texts

    def stretch_audio_segment(self, original_duration_sec, audio_response):
        """
        Stretch TTS audio to match original duration.
        """
        # Decode base64 audio
        audio_bytes = base64.b64decode(audio_response.audios[0])

        # Load audio with soundfile to handle in-memory files
        with io.BytesIO(audio_bytes) as bio:
            y, sr = sf.read(bio)

        # Calculate current TTS duration
        tts_duration_sec = len(y) / sr

        # Handle zero/negative duration edge cases
        if tts_duration_sec <= 0 or original_duration_sec <= 0:
            return AudioSegment.silent(duration=original_duration_sec * 1000)

        # Compute stretch rate (inverse of RubberBand's approach)
        rate = tts_duration_sec / original_duration_sec

        # Apply librosa's time_stretch (mono only)
        if y.ndim > 1:
            y = librosa.to_mono(y.T)

        y_stretched = librosa.effects.time_stretch(y, rate=rate)

        # Convert back to AudioSegment
        with io.BytesIO() as output_bio:
            sf.write(output_bio, y_stretched, sr, format='WAV')
            output_bio.seek(0)
            return AudioSegment.from_wav(output_bio)

    def convert_text_to_speech(
            self,
            texts,
            target_language_code,
            model: str ="bulbul:v2",
            speaker: str ="karun",
            pitch: int = 0.1,
            pace: int = 1.0,
            loudness: int = 1.0,
            speech_sample_rate: str = 24000,
            enable_preprocessing: bool = True
        ):

        """
        This function is used to convert the given text to target language audio

        Return:
            combined_stretched_audio : Audio segment where each portion has been time-stretched to match its target duration.
            raw_audio : Concatenated raw audio output directly from the TTS engine.
        """

        combined_stretched_audio = AudioSegment.empty()
        raw_audio = AudioSegment.empty()

        for i, text in enumerate(texts):
            # # Set parameters based on text content
            # if "!" in text:
            #     # Exclamation: higher pitch, faster pace, louder
            #     pitch = 0.2
            #     pace = 1.0
            #     loudness = 1.2
            # elif "?" in text:
            #     # Question: slightly higher pitch, slower pace
            #     pitch = 0.1
            #     pace = 0.7
            #     loudness = 1.0
            # else:
            #     # Neutral/default
            #     pitch = -0.1
            #     pace = 0.8
            #     loudness = 1.0

            audio = self.client.text_to_speech.convert(
                text=text,
                target_language_code=target_language_code,
                model=model,
                speaker=speaker,
                pitch=pitch,
                pace=pace,
                loudness=loudness,
                speech_sample_rate=speech_sample_rate,
                enable_preprocessing=enable_preprocessing
            )

            audio_segment = AudioSegment.from_file(io.BytesIO(base64.b64decode(audio.audios[0])), format="wav")
            raw_audio += audio_segment

            # stretched_audio = self.stretch_audio_segment(seg['duration'], audio)
            # combined_stretched_audio += stretched_audio
            print(f"Processed batch {i + 1}")

        return combined_stretched_audio, raw_audio

In [ ]:
API_SUBSCRIPTION_KEY = userdata.get("SARVAM_SUBSCRIPTION_KEY")

client = SarvamClientAPI(API_SUBSCRIPTION_KEY)

## English-to-Malayalam Text Translation using Sarvam Mayura API

In [ ]:
translated_texts = client.translate_text(
    sentences,
    source_lang="en-IN",
    target_lang="ml-IN",
    gender="Male",
    mode="code-mixed",
    output_script="spoken-form-in-native",
    enable_preprocessing=True
)

# pprint(translated_texts)

Processed batch 1 ...
Processed batch 2 ...
Processed batch 3 ...
Processed batch 4 ...
Processed batch 5 ...
Processed batch 6 ...
Processed batch 7 ...
Processed batch 8 ...
Processed batch 9 ...
Processed batch 10 ...
Processed batch 11 ...
Processed batch 12 ...
Processed batch 13 ...
Processed batch 14 ...
Processed batch 15 ...
Processed batch 16 ...
Processed batch 17 ...
Processed batch 18 ...
Processed batch 19 ...
Processed batch 20 ...
Processed batch 21 ...
Processed batch 22 ...
Processed batch 23 ...
Processed batch 24 ...
Processed batch 25 ...
Processed batch 26 ...
Processed batch 27 ...
Processed batch 28 ...
Processed batch 29 ...
Processed batch 30 ...
Processed batch 31 ...
Processed batch 32 ...
Processed batch 33 ...
Processed batch 34 ...
Processed batch 35 ...
Processed batch 36 ...
Processed batch 37 ...
Processed batch 38 ...
Processed batch 39 ...
Processed batch 40 ...
Processed batch 41 ...
Processed batch 42 ...
Processed batch 43 ...
Processed batch 44 .

In [ ]:
# def remove_commas_from_texts(data_list):
#     for item in data_list:
#         # Remove all commas from the 'text' string
#         item['text'] = item['text'].replace(',', '')
#     return data_list

# updated_texts = remove_commas_from_texts(mal_texts)
# updated_texts

import re

def reduce_question_marks(texts):
    """
    Replaces sequences of multiple question marks with a single question mark.

    Args:
        text : The input string list.

    Returns:
        str: Modified string with only one '?' per sequence.
    """
    preproc_text = []
    for text in texts:
        text = re.sub(r'\?{2,}', '?', text)
        text = re.sub(r'ക്വസ്റ്റ്യൻ മാർക്ക്', '', text)
        preproc_text.append(text)
    return preproc_text

translated_texts = reduce_question_marks(translated_texts)
translated_texts

['ഒരു കൾച്ചറിൻ്റെ യുണിക് ഫിംഗർപ്രിൻ്റ് ആണ് ലാംഗ്വേജ്, അല്ലേ?',
 'അപ്പോൾ ഞങ്ങൾ എ ഐ ഫോർ ഭാരത് കൺസിഡർ ചെയ്യുമ്പോൾ, എ ഐ ഫോർ ഭാരത് എന്താണെന്ന് സീരിയസ്ലി ആയിട്ട് ആലോചിച്ചു.',
 'ഇന്ത്യയിൽ നിന്നുള്ള ഇമേജസുമായി നന്നായി പ്രവർത്തിക്കാൻ ഒരു കമ്പ്യൂട്ടർ വിഷൻ അൽഗോരിതം അഡ്ജസ്റ്റ് ചെയ്യേണ്ടി വരും.',
 'പക്ഷേ ലാംഗ്വേജസിന് വേണ്ടി താങ്കൾ കൂടുതൽ ചേഞ്ചസ് വരുത്തേണ്ടതുണ്ട്.',
 'ഡീപ് ലേണിങ് എങ്ങനെ പോപ്പുലർ ആയി എന്നുള്ള കഥ വളരെ ഫാസിനറ്റിങ് ആണ്. ഡീപ് ലേണിങ് മൊമെന്റം നേടുന്ന ഒരു സമയത്തായിരുന്നു അത്.',
 'അത് മുൻപത്തെ എ ഐ ജനറേഷനോട് സമാനമായിരുന്നു.',
 'ഇത് ജനറേറ്റീവ് എ ഐ ആണ്.',
 'പിന്നെ ഞങ്ങൾ ഡീപ് ലേണിങ്ങിൽ കോഴ്സസ് ഒന്നിച്ച് പഠിപ്പിക്കാൻ തുടങ്ങി.',
 'ഈ ഏരിയ മെയിൻലി തിയററ്റിക്കൽ ആയിരുന്നത് ഇപ്പോൾ പ്രാക്ടിക്കൽ ആപ്ലിക്കേഷനുകളിൽ കൂടുതൽ ഫോക്കസ് ചെയ്യുന്നു എന്നാണല്ലോ ഞങ്ങൾ ശ്രദ്ധിച്ചത്. ഈ ഷിഫ്റ്റിൽ ലാർജ് സ്കെയിലിൽ കാര്യങ്ങൾ ചെയ്യുന്നതും ഈ മോഡൽസ് ഡെവലപ്പ് ചെയ്യുന്നതും ഉൾപ്പെടുന്നു. ഇത് ഒരു ഇൻട്രിഗിങ് എക്സ്പീരിയൻസ് ആയിരുന്നു.',
 'അപ്പോൾ ഞങ്ങൾ ടീച്ച് ചെയ്യാൻ തുടങ്ങി.',
 'ഈ ഐഡിയ ആക്ഷനിലേക്ക് കൊണ്ടുവരാൻ ആഗ്രഹിച്ച ഒരുപാട് ആളുകൾ

## Text-to-Speech using Sarvam BulbulV2

In [ ]:
stretched_audio, raw_audio = client.convert_text_to_speech(
    translated_texts,
    target_language_code="ml-IN",
    model="bulbul:v2",
    speaker="karun",
    pitch=-0.1,
    pace=1.164,
    loudness=1.4,
    speech_sample_rate=24000,
)
# stretch_audio is speed controlled to fit the video duration
# raw_audio is the raw data got from TTS api, not speed controlled


Processed batch 1
Processed batch 2
Processed batch 3
Processed batch 4
Processed batch 5
Processed batch 6
Processed batch 7
Processed batch 8
Processed batch 9
Processed batch 10
Processed batch 11
Processed batch 12
Processed batch 13
Processed batch 14
Processed batch 15
Processed batch 16
Processed batch 17
Processed batch 18
Processed batch 19
Processed batch 20
Processed batch 21
Processed batch 22
Processed batch 23
Processed batch 24
Processed batch 25
Processed batch 26
Processed batch 27
Processed batch 28
Processed batch 29
Processed batch 30
Processed batch 31
Processed batch 32
Processed batch 33
Processed batch 34
Processed batch 35
Processed batch 36
Processed batch 37
Processed batch 38
Processed batch 39
Processed batch 40
Processed batch 41
Processed batch 42
Processed batch 43
Processed batch 44
Processed batch 45
Processed batch 46
Processed batch 47
Processed batch 48


In [ ]:
# Make sure the directory exists before downloading
os.makedirs(OUTPUT_AUDIO_PATH, exist_ok=True)

# Export the combined audio to a file
# stretched_audio.export(OUTPUT_AUDIO_FILE, format="wav")
raw_audio.export(RAW_OUTPUT_AUDIO_FILE, format="wav")

<_io.BufferedRandom name='/content/output_data/raw_output_audio.wav'>

In [ ]:
from IPython.display import Audio, display

# Display separated tracks
display(
    # Audio(OUTPUT_AUDIO_FILE, autoplay=False),
    Audio(RAW_OUTPUT_AUDIO_FILE, autoplay=False)
)

## Reassembling and Merging Audio and Video

### Removing audio from the original input video

In [ ]:
%ls
!ffmpeg -i input_data/input_video.mp4 -an -c:v copy muted_video.mp4 -y

drive/       output_data/         pretrained_models/  text_data/
input_data/  preprocessed_audio/  sample_data/
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --en

In [ ]:
# Display the input video
Video('muted_video.mp4', width=640, height=360, embed=True)

### Adding the dubbed audio to the video

In [ ]:
!ffmpeg -i muted_video.mp4 -i output_data/raw_output_audio.wav -c:v copy -map 0:v:0 -map 1:a:0 -shortest output_wo_bgm.mp4 -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

### Adding the accompaniment audio to the final output video

In [ ]:
!ffmpeg -i output_wo_bgm.mp4 -i preprocessed_audio/accompaniment.wav -filter_complex "[0:a][1:a]amix=inputs=2:duration=first:dropout_transition=3" -c:v copy -map 0:v:0 -shortest final_output.mp4 -y

In [ ]:
# # Display the final output audio
Audio(RAW_OUTPUT_AUDIO_FILE, autoplay=False)

In [ ]:
# Display the final output video
Video('final_output.mp4', width=640, height=360, embed=True)

# AUDIO SEEMS LAGGY AS IT WAS STRETCHED TO MEET THE VIDEO LENGTH, PURE DUBBED AUDIO IS AVAILABLE IN THE ABOVE CODE BLOCK

Buffered data was truncated after reaching the output size limit.